# Convert Raster Data to DGGS Zones

In [1]:
import os

import folium
import json
import pystac_client
import dggrid4py
import geopandas as gpd
import numpy as np
import shapely
import rasterio as rio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from tqdm.autonotebook import tqdm

from vgrid.conversion.dggs2geo.dggrid2geo import dggrid2geo
from vgrid.conversion.raster2dggs import raster2a5, raster2dggrid, raster2rhealpix, raster2h3
from vgrid.conversion.raster2dggs.raster2dggrid import get_nearest_dggrid_resolution
from vgrid.conversion.latlon2dggs import latlon2dggrid
from vgrid.utils.geometry import geodesic_dggs_metrics
from vgrid.utils.io import (
    convert_to_output_format,
    create_dggrid_instance,
    download_file,
    validate_dggrid_type,
    validate_dggrid_resolution,
)

/tmp/ipykernel_62845/876217188.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# [optional] extra dependencies for nicer notebook progress bars if not available
!pip install -U tqdm ipywidgets jupyterlab_widgets

## Identify the Study Area of Interest

In [3]:
with open("../manitoba-winnipeg_study_area/inputs/manitoba-winnipeg_study_area.geojson") as f:
    data = f.read()
    poly = json.loads(data)
    geom = shapely.from_geojson(data)

map = folium.Map(location=[geom.centroid.y, geom.centroid.x], zoom_start=8)
folium.GeoJson(poly).add_to(map)
map

## Find RCM ARD Data from EODMS matching AOI

In [4]:
client = pystac_client.Client.open("https://www.eodms-sgdot.nrcan-rncan.gc.ca/stac/")
result = client.search(
    collections=["rcm-ard"],
    intersects=geom,
    datetime=["2018-01-01", "2026-01-01"],
    limit=100,
)
result.matched()

87

In [5]:
item = next(result.items())
item

<Item id=7c1940f7-ebd6-416a-a0cc-8e2d3c532350>

## Define DGGRID Instance (optional)

This is needed only if you want to use `raster2dggrid` conversion function.

⚠️ Below code requires https://github.com/allixender/dggrid4py/pull/26 (i.e.: `DGGRIDv8`) to resolve DGGRID system indexing issues.

In [6]:
DGGRID_EXE = os.getenv("DGGRID_EXE", "../../DGGRID/build/src/apps/dggrid/dggrid")
# dggrid_instance = create_dggrid_instance(DGGRID_EXE) # FIX: create ourselves to avoid legacy errors and add GDAL support
dggrid_instance = dggrid4py.DGGRIDv8(
    executable=DGGRID_EXE,
    working_dir='.',
    capture_logs=True,
    has_gdal=True,
    tmp_geo_out_legacy=False,
    silent=False,
    debug=False,
)

In [19]:
def raster_reproject(raster_path, dst_crs="EPSG:4326") -> str:
    reproject_name, reproject_ext = os.path.splitext(os.path.basename(raster_path))
    reproject_name = f"{reproject_name}_{dst_crs.replace(':', '')}"
    reproject_path = os.path.join(os.path.dirname(raster_path), f"{reproject_name}{reproject_ext}")
    if os.path.isfile(reproject_path):
        return reproject_path

    with rio.open(raster_path) as src:
        if src.crs == dst_crs:
            return raster_path

        transform, width, height = calculate_default_transform(src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        with rio.open(reproject_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rio.band(src, i),
                    destination=rio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest,
                )
    return reproject_path


def raster2dggs_custom(
    dggrid_instance,
    dggs_type: str,
    raster_path,
    resolution: int | None = None,
    output_format: str = "gpd",
    output_cell_path: str | None = None,  # raster path by default if not specified
    **output_kwargs,
):
    dggs_type = validate_dggrid_type(dggs_type)
    raster_path = raster_reproject(raster_path)  # pre-reproject to reduce time for each iter of dggrid
    output_cell_path = output_cell_path or os.path.dirname(raster_path)

    # Auto-select resolution if not provided
    if resolution is None:
        cell_size, resolution = get_nearest_dggrid_resolution(dggrid_instance, dggs_type, raster_path)
        print(f"Cell size: {cell_size} m2")
        print(f"Nearest {dggs_type.upper()} resolution determined: {resolution}")
    else:
        resolution = validate_dggrid_resolution(dggs_type, resolution)

    # Open the raster file to get metadata and data
    src = rio.open(raster_path)

    # collect cells
    raster_geom = shapely.geometry.box(*list(src.bounds))
    cells = dggrid_instance.grid_cell_polygons_for_extent(
        dggs_type,
        resolution=resolution,
        clip_geom=raster_geom,
        **output_kwargs
    )
    cell_count = len(cells)
    print(f"Matched cells: {cell_count} (res={resolution}) in {raster_geom}")

    properties = []

    # Collect band values during the pixel scan, storing the first sample per DGGRID cell
    for cell in tqdm(
        cells.itertuples(index=False),
        desc="Converting raster to DGGRID",
        unit=" cells",
        total=cell_count,
    ):
        cell_id = cell.name if hasattr(cell, "name") else cell.global_id
        cell_geom = cell.geometry
        cell_data, cell_transform = mask(src, [cell_geom], crop=True)
        cell_meta = src.meta.copy()
        cell_meta.update({
            "driver": "GTiff",
            "transform": cell_transform,
            "crs": src.crs,
            "height": cell_data.shape[1],
            "width": cell_data.shape[2],
        })

        cell_valid_data = cell_data[cell_data != src.nodata]
        if len(cell_valid_data) == 0:
            continue  # skip empty cells

        output_cell_file = os.path.join(output_cell_path, f"cell-{cell_id}.tif")
        with rio.open(output_cell_file, "w", **cell_meta) as cell_f:
            cell_f.write(cell_data)

        cell_mean = np.mean(cell_valid_data)
        cell_stddev = np.std(cell_valid_data)
        cell_median = np.median(cell_valid_data)
        cell_minimum = np.min(cell_valid_data)
        cell_maximum = np.max(cell_valid_data)
        cell_pixel_count = len(cell_valid_data)

        # Build GeoDataFrame as the base
        try:
            # Get cell metrics
            #if isinstance(cell_polygon, gpd.GeoDataFrame) and not cell_geom.empty:
            centroid_lat, centroid_lon, avg_edge_len, cell_area, cell_perimeter = geodesic_dggs_metrics(
                cell_geom, len(cell_geom.exterior.coords) - 1
            )
            # else:
            #     # Fallback metrics if geometry conversion fails
            #     centroid_lat, centroid_lon, avg_edge_len, cell_area, cell_perimeter = 0, 0, 0, 0, 0

            cell_props = {
                f"dggrid_{dggs_type}": cell_id,
                "resolution": resolution,
                "center_lat": centroid_lat,
                "center_lon": centroid_lon,
                "avg_edge_len": avg_edge_len,
                "cell_area": cell_area,
                "cell_perimeter": cell_perimeter,
                "cell_minimum": cell_minimum,
                "cell_maximum": cell_maximum,
                "cell_mean": cell_mean,
                "cell_median": cell_median,
                "cell_stddev": cell_stddev,
                "cell_pixel_count": cell_pixel_count,
                "geometry": cell_geom # if isinstance(cell_polygon, gpd.GeoDataFrame) and not cell_polygon.empty else None,
            }
            properties.append(cell_props)
        except Exception:
            continue

    src.close()

    # Build GeoDataFrame
    gdf = gpd.GeoDataFrame(properties, geometry="geometry", crs="EPSG:4326")

    # Use centralized output utility
    base_name = os.path.splitext(os.path.basename(raster_path))[0]
    output_name = f"{base_name}2dggrid" if output_format is not None else None
    return convert_to_output_format(gdf, output_format, output_name)

## Convert RCM ARD Data to DGGS

#### NOTE

⚠️ Requires patch: https://github.com/opengeoshub/vgrid/pull/57 if running the `vgrid` "raster2..." native operations
on non-geometric CRS (i.e.: PROJCRS) rasters.


In [ ]:
output_cell_stats_formats = ["geojson", "parquet"]
output_cell_data_format = ".tif"
dggs_types_res = [
    # ("H3", lambda: _src, _res, _out_fmt, **__: raster2h3(_src, _res, _out_fmt), [7, 8]),  # H3 L8 hexagon dimension ~= ISEA7H L9
    # ("A5", lambda: _src, _res, _out_fmt, **__: raster2a5(_src, _res, _out_fmt), [9]),
    # ("rHEALPix", lambda: _src, _res, _out_fmt, **__: raster2rhealpix(_src, _res, _out_fmt), [8, 9]),
    # NOTE: raster2dggrid
    #   wrapper dispatches to the underlying "dggrid" tool that provide additional DGGRS
    #   IGEO7 = ISEA7H with 7Z address representation (https://dggrid4py.readthedocs.io/en/latest/usage.html)
    # WARNING:
    #   use dggrid BIN_POINT_VALS directly or another custom method instead of 'raster2dggrid'
    #   raster2dggrid is very slow since it computes every lat/lon=>cellId for every (x,y) pixel sequentially
    #   it also reprojects each of those (lat,lon) coordinates one by one
    #   raster2dggrid also incorrectly picks any first cellId match and drops all other values within that same cell
    #   depending on the cell size/resolution, this first value might not be representative at all!
    # (
    #     "IGEO7",
    #     lambda *_, **__: raster2dggrid(dggrid_instance, "IGEO7", *_, dggrid_input_address_type="Z7_STRING", **__),
    #     [8, 9]
    # ),
    (
        "IGEO7",
        lambda *_, **__: raster2dggs_custom(
            dggrid_instance,
            "ISEA7H",
            *_,
            # WARNING: following requires new DGGRID v8
            #dggrid_output_address_type="Z7_STRING",
            output_cell_label_type="OUTPUT_ADDRESS_TYPE",
            output_address_type="HIERNDX",
            output_hier_ndx_system="Z7",
            output_hier_ndx_form="DIGIT_STRING",
            **__,
        ),
        [7, 8]  # pre-computed res=15 (~10m) closest to ~20m rasters retrieved from STAC
    ),
]

dggs_bar = tqdm(dggs_types_res, desc="Processing DGGS Types")
for dggrs, dggrs_builder, dggrs_resolutions in dggs_bar:
    dggs_bar.set_postfix(dggrs=dggrs)
    res_bar = tqdm(dggrs_resolutions, desc="Processing Resolutions", leave=False, total=len(dggrs_resolutions))
    for resolution in res_bar:
        res_bar.set_postfix(dggrs=dggrs, resolution=resolution)
        stac_items_bar = tqdm(result.items(), desc="Processing STAC Items", leave=False, total=result.matched())
        for item in stac_items_bar:
            stac_items_bar.set_postfix(dggrs=dggrs, resolution=resolution, item=item.id)
            date = item.properties["datetime"][:10]  # YYYY-MM-DD
            for band in ["rl", "rr"]:
                src_uri = item.assets[band].href
                src_name = os.path.basename(src_uri)
                res_bar.set_postfix(dggrs=dggrs, resolution=resolution, item=src_name)

                src_dir = os.path.join("./outputs", f"manitoba_rcm_ard", "src", date, band)
                src_path = os.path.join(src_dir, src_name)
                out_dir = os.path.join("./outputs", "manitoba_rcm_ard", dggrs, f"L{resolution}", date, band)
                os.makedirs(out_dir, exist_ok=True)
                out_files = {
                    fmt: os.path.join(out_dir, f"{os.path.splitext(src_name)[0]}.{fmt}")
                    for fmt in output_cell_stats_formats
                }
                if all(os.path.isfile(p) for p in out_files.values()):
                    print(f"  Skipping... All output files already exist: {list(out_files.values())}")
                    continue

                print(f"Converting to {dggrs}-L{resolution} [band={band}]. Saving to [{out_dir}]...")
                if not os.path.isfile(src_path):
                    os.makedirs(src_dir, exist_ok=True)
                    download_file(src_uri, src_path)
                out_df = dggrs_builder(
                    src_path,
                    resolution=resolution,
                    output_format="gpd",  # GeoPandas Dataframe
                    output_cell_path=out_dir,  # only custom function
                )
                # save results
                for fmt, out_path in out_files.items():
                    convert_to_output_format(out_df, output_format=fmt, output_name=out_path)

Processing DGGS Types:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Resolutions:   0%|          | 0/2 [00:00<?, ?it/s]

Processing STAC Items:   0%|          | 0/87 [00:00<?, ?it/s]